In [ ]:
# Import necessary packages
# Requirements:
# (1) pytorch 
# (2) numpy 
# (3) matplotlib
# (4) pandas

import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from ttictoc import tic, toc
import csv

print(torch.cuda.is_available())
print(os.getcwd())

In [ ]:
# Define speeds (m/s) corresponding to 0-40 Hz settings in wind tunnel
### (Just for record-keeping)
hzArray = np.array((0, 5, 10, 15, 20, 25, 30, 35, 40))
speedArray = np.array((0.00, 1.26363735, 1.58562983, 2.07066356, 2.571993, 3.18291372, 3.75322345, 4.33626595, 4.91413509))
###

In [ ]:
class CrossWireDataset(Dataset):
    '''
    Dataset class for pytorch-based learning tailored to crosswire model training. This method 
    essentially is feature learning of a specific, reduced set of features from the sensor readings, 
    namely: 

    [Input Features]
       --- The maximal (absolute) voltage reading (voltage)
       --- The index of the maximal (absolute) voltage reading (integer, {1-6})
       --- The (regularized) ratio of the adjacent sensors (voltage/voltage)
    [Predictions]
       --- The gust speed (m/s)
       --- The gust incident angle (radians)

    '''
    def __init__(self, magFile, angFile, readingsFile, transform=None, target_transform=None):
        # Construct the labels
        tmpMag = pd.read_csv(magFile)
        tmpAng = pd.read_csv(angFile)
        self.mags = torch.Tensor(tmpMag.to_numpy())
        self.angs = torch.Tensor(tmpAng.to_numpy())
        
        # Construct the features and place them into readings array(X). 
        tmpReadings = pd.read_csv(readingsFile)
        tmpReadings = tmpReadings.to_numpy()
        print(tmpReadings.shape)
        LL = tmpReadings.shape[0]
        tmpReadings2 = np.zeros((LL, 3))
        for k in range(LL):
            tmpReadings2[k, 0] = np.max(np.abs(tmpReadings[k, :]))
            tmpReadings2[k, 1] = np.argmax(np.abs(tmpReadings[k, :]))
            tt = int(tmpReadings2[k,1])
            tmpReadings2[k, 2] = np.abs(tmpReadings[k, (tt-1)%6])/(np.abs(tmpReadings[k, (tt+1)%6]) + 0.05)
        
        self.readings = torch.Tensor(tmpReadings2)
        
        # Incorporate the transforms as needed
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.mags)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        mag = self.mags[idx]
        ang = self.angs[idx]
        label = torch.cat((mag, ang), 0)
        
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            label = self.target_transform(label)

        return reading, label

In [ ]:
class WindMagDataset(Dataset):
    '''
    Dataset class for pytorch-based learning for gust magnitude data training. This method 
    learns directly from the sensor readings (voltages) to predict gust speed (m/s). 
    [Inputs]
       --- The sensor readings (voltages)
    [Predictions]
       --- The gust speed (m/s)
    '''
    def __init__(self, magFile, readingsFile, transform=None, target_transform=None):
        tmpMag = pd.read_csv(magFile)
        tmpReadings = pd.read_csv(readingsFile)
        self.mags = torch.Tensor(tmpMag.to_numpy())
        self.readings = torch.Tensor(tmpReadings.to_numpy())
        
        # Incorporate the transforms as needed
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.mags)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        label = self.mags[idx]
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            label = self.target_transform(label)

        return reading, label

class WindAngDataset(Dataset):
    '''
    Dataset class for pytorch-based learning for gust angle data training. This method 
    learns directly from the sensor readings (voltages) to predict gust incidence angle (radians). 
    [Inputs]
       --- The sensor readings (voltages)
    [Predictions]
       --- The gust angle (rad)
    '''
    def __init__(self, angFile, readingsFile, transform=None, target_transform=None):
        tmpAng = pd.read_csv(angFile)
        tmpReadings = pd.read_csv(readingsFile)
        self.angs = torch.Tensor(tmpAng.to_numpy())
        self.readings = torch.Tensor(tmpReadings.to_numpy())
        
        # Incorporate the transforms as needed
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.angs)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        label = self.angs[idx]
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            label = self.target_transform(label)

        return reading, label

class WindAngTrigDataset(Dataset):
    '''
    Dataset class for pytorch-based learning for gust angle data training. This method 
    learns directly from the sensor readings (voltages) to predict gust incidence angle (radians). 
    [Inputs]
       --- The sensor readings (voltages)
    [Predictions]
       --- The gust angle (rad)
    '''
    def __init__(self, angFile, readingsFile, transform=None, target_transform=None):
        tmpAng = pd.read_csv(angFile)
        tmpReadings = pd.read_csv(readingsFile)
        numpyAngs = tmpAng.to_numpy()
        self.angs = torch.Tensor(np.array((np.cos(numpyAngs), np.sin(numpyAngs))))
        self.readings = torch.Tensor(tmpReadings.to_numpy())
        
        # Incorporate the transforms as needed
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.angs[:, 0])
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        label = self.angs[idx, :]
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            label = self.target_transform(label)

        return reading, label

In [ ]:
class NeuralNetwork(nn.Module):
    '''
    A general/generic Neural Network model class for use with Pytorch. 
    
    TODO: include layer widths, types, and nonlinearities as inputs and dynamically allocate
          --- this will allow for custom classes rather than the clunky "if" statement used here. 
    '''
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        
        
    def forward(self, x):
        pass

In [ ]:
class CrosswireNet(NeuralNetwork):
    '''
    A general/generic Neural Network model class for use with Pytorch. 
    
    TODO: include layer widths, types, and nonlinearities as inputs and dynamically allocate
          --- this will allow for custom classes rather than the clunky "if" statement used here. 
    '''
    def __init__(self):
        super(CrosswireNet, self).__init__()
        
        self.linear_relu_stack = nn.Sequential(nn.Linear(3, 25),
                                               nn.ReLU(),
                                               nn.Linear(25, 15),
                                               nn.ReLU(),
                                               nn.Linear(15, 2),
                                               )
    def forward(self, x):
        # Method to propagate input (reading) through the network to get a prediction. 
        # Terminology is clunky because this is adapted from a classification example, hence 
        # the use of 'logits' even though we are doing regression.
        
        # TODO -- tidy up variable names, usage, etc (see above)
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


class AngleNet(NeuralNetwork):
    '''
    A general/generic Neural Network model class for use with Pytorch. 
    
    TODO: include layer widths, types, and nonlinearities as inputs and dynamically allocate
          --- this will allow for custom classes rather than the clunky "if" statement used here. 
    '''
    def __init__(self, geom=6, trigValue=1):
        super(CrosswireNet, self).__init__()
        
        k1 = int(geom*8)
        k2 = int(k1/2 + 5)
        self.linear_relu_stack = nn.Sequential(nn.Linear(geom, k1),
                                               nn.ReLU(),
                                               nn.Linear(k1, k2),
                                               nn.ReLU(),
                                               nn.Linear(k2, trigValue),
                                               )
        
    def forward(self, x):
        # Method to propagate input (reading) through the network to get a prediction. 
        # Terminology is clunky because this is adapted from a classification example, hence 
        # the use of 'logits' even though we are doing regression.
        
        # TODO -- tidy up variable names, usage, etc (see above)
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
def train_loop(dataloader, model, optimizer, epochNum, seedNum, loss_fn=nn.L1Loss(), verbose=True, batch_size=180, writePath=None):
    """
    Loop for training a 'model' (class NeuralNetwork) on data stored in 'dataloader,' using loss function
    'loss_fn' and optimizer method 'optimizer'
    
    [Inputs]
    dataloader    -- type DataLoader    -- Pytorch DataLoader object to facilitate training/testing data storage
                                           to interface with pytorch optimization and training modules
                                           
    model         -- type NeuralNetwork -- Pytorch NeuralNetwork object to facilitate training/testing of speed and 
                                           angle prediction for FlowDrone
                                           
    epochNum      -- type int           -- Current epoch number to track training loss
    
    seedNum       -- type int           -- Seed of the current run (to average over to demonstrate convergence)
    
    loss_fn       -- type torch.nn loss -- Pytorch loss function (in nn library) for training the speed/angle predictor
                                           for FlowDrone. Defaults to nn.MSELoss() because we are regressing real-valued
                                           variables. 
                                           
    optimizer     -- type torch.optim   -- Pytorch optimizer for ANN weight updates. Normally will use ADAM unless there
                                           is a compelling reason to deviate. 
    
    verbose       -- type Boolean       -- Toggles printing of training loss during training. Default is TRUE. 
    
    writePath     -- type String        -- If present, a path to write to a csv file in order 
    [Outputs]
    
    None

    """
    size = len(dataloader.dataset)
    f = open(writePath+'trainCost'+str(seedNum)+'.csv', 'a')
    writer = csv.writer(f)
    
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        '''
        for k in range(pred.shape[0]):
            if ((pred[k, -1] - y[k, -1]) >= math.pi):
                while ((pred[k, -1] - y[k, -1]) >= math.pi):
                    pred[k, -1] -= 2.0*math.pi
            elif ((pred[k, -1] - y[k, -1]) <= -math.pi):
                pred[k, -1] = (pred[k,-1])%(2.0*math.pi)
        '''
        loss = loss_fn.forward(y, pred)
        loss_average = (loss/pred.shape[0]).cpu().detach().numpy()
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 200 == 199 or (batch==0 and epochNum==1):
            if verbose:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            if writePath is None:
                pass
            else: 
                writer.writerow(np.array([(batch + (np.ceil(size/batch_size)*epochNum)), 180.0*loss_average/math.pi]))
                

    # close the file
    f.close()
        


def test_loop(dataloader, model, epochNum, seedNum, loss_fn=nn.L1Loss(), lastLoop=False, writePath=None):
    """
    Loop for test a 'model' (class NeuralNetwork) on data stored in 'dataloader,' using loss function
    'loss_fn.'
    
    [Inputs]
    dataloader    -- type DataLoader    -- Pytorch DataLoader object to facilitate training/testing data storage
                                           to interface with pytorch optimization and training modules
                                           
    model         -- type NeuralNetwork -- Pytorch NeuralNetwork object to facilitate training/testing of speed and 
                                           angle prediction for FlowDrone

    epochNum      -- type Int           -- Current epoch
    
    loss_fn       -- type torch.nn loss -- Pytorch loss function (in nn library) for training the speed/angle predictor
                                           for FlowDrone. Defaults to nn.MSELoss() because we are regressing real-valued
                                           variables. 
                                           
    lastLoop      -- type Boolean       -- Whether we are on the last epoch (for histogram information)
    
    writePath     -- type String        -- File to write to
    [Outputs]
    
    None

    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    batch_size = dataloader.batch_size
    hists = False
    
    if lastLoop:
        if (num_batches == size):
            print('On the last loop!')
            errs = np.zeros((size, 2))
            idxVal = 0
            hists = True
        
    test_loss = 0.0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            '''
            for k in range(pred.shape[0]):
                if ((pred[k, -1] - y[k, -1]) >= math.pi):
                    while ((pred[k, -1] - y[k, -1]) >= math.pi):
                        pred[k, -1] -= 2.0*math.pi
                elif ((pred[k, -1] - y[k, -1]) <= -math.pi):
                    pred[k, -1] = (pred[k,-1])%(2.0*math.pi)
            '''
            if hists:
                errs[idxVal, :] = np.array([y, loss_fn.forward(y, pred)])
                # errs[idxVal, :] = np.array([y, loss_fn(y, pred).item()])
                test_loss += errs[idxVal, 1]
                idxVal += 1
            else:
                test_loss += loss_fn.forward(y, pred)/pred.shape[0]

    test_loss /= num_batches
    print(f"Avg loss (rad): {test_loss:>8f}")
    print(f"Avg error (deg): {(test_loss*180.0/math.pi):>8f} \n")
    
    if writePath is None:
        pass
    else:
        # open the file
        f = open(writePath+'testCost'+str(seedNum)+'.csv', 'a')
        writer = csv.writer(f)
        
        # write out the relevant cost
        writer.writerow(np.array([epochNum, test_loss*180.0/math.pi]))
                
        # close the file
        f.close()
    
    if hists:
        np.savetxt(writePath+'testErrs'+str(seedNum)+'.csv', errs, delimiter=',')
        return test_loss, errs
    else: 
        return test_loss

In [ ]:
def makeDataset(dataSetType=2, geometryVal=3, compFlag=True, compString=None, N=1, epochs0=15):
    # Make dataset
    # Change this as desired in {1, 2, 3, 4, 5}
    #
    # INDEX: 
    #   --- (1) Sparse wind magnitudes                         [OLD]
    #   --- (2) Sparse wind angles (10-degree increments)      [OLD]
    #   --- (3) Dense Crosswire Model 
    #   --- (4) Dense Wind Magnitudes 
    #   --- (5) Dense Incidence Angles (2-degree increments)
    
    ### dataSetType = 2

    # 3=Triangle, 4=Square, 5=Pentagon, 6=Hexagon
    ### geometryVal = 3
    ### compFlag=True
    ### N = 1                 # Number of sequentially averaged data points

    if geometryVal == 3:
        geomPath='tri/'
    elif geometryVal == 4:
        geomPath='squ/'
    elif geometryVal == 5:
        geomPath='pent/'
    elif geometryVal == 6:
        geomPath='hex/'
    else:
        raise ValueError('Geometry must be in {3, 4, 5, 6}')

    testPathBase = 'compVal' if compString is None else compString
    
    trainLabelPath = 'compTrain/'
    testLabelPath = testPathBase+'/'
    trainPath='compTrain_N'+str(N)+'/'
    testPath= testPathBase+'_N'+str(N)+'/'       # Set to validation data for network/hyperparameter optimization, else test data

    # Don't change these; the 'if' statements take care of them
    # Set network parameters in NeuralNetwork class
    fullAnglesVal = False
    crosswireVal = False

    if dataSetType==1:
        if compFlag:
            trainY = trainLabelPath+'mags.csv'
            trainX = trainPath+geomPath+'readings.csv'
            testY = testLabelPath+'mags.csv'
            testX = testPath+geomPath+'readings.csv'
        else:
            trainY = 'MagTrain/mags.csv'
            trainX = 'MagTrain/readings.csv'
            testY = 'MagTest/mags.csv'
            testX = 'MagTest/readings.csv'

        training_data = WindMagDataset(trainY, trainX, transform=None)
        testing_data = WindMagDataset(testY, testX, transform=None)
        epochs = epochs0

    elif dataSetType==2:
        if compFlag:
            trainY = trainLabelPath+'angsrad.csv'
            trainX = trainPath+geomPath+'readings.csv'
            testY = testLabelPath+'angsrad.csv'
            testX = testPath+geomPath+'readings.csv'
            fullAnglesVal = True

        else:
            trainY = 'MagTrain/angsrad.csv'
            trainX = 'MagTrain/readings.csv'
            testY = 'MagTest/angsrad.csv'
            testX = 'MagTest/readings.csv'

        training_data = WindAngDataset(trainY, trainX, transform=None)
        testing_data = WindAngDataset(testY, testX, transform=None)
        epochs = epochs0

    elif dataSetType==3:
        trainY1 = 'CrossTrain/crossmags.csv'
        trainY2 = 'CrossTrain/crossangsrad.csv'
        trainX = 'CrossTrain/crossreadings.csv'
        testY1 = 'CrossTest/crossmags.csv'
        testY2 = 'CrossTest/crossangsrad.csv'
        testX = 'CrossTest/crossreadings.csv'

        training_data = CrossWireDataset(trainY1, trainY2, trainX, transform=None)
        testing_data = CrossWireDataset(testY1, testY2, testX, transform=None)
        epochs = 2*epochs0

        crosswireVal = True

    elif dataSetType==4:

        trainY = 'CrossTrain/crossmags.csv'
        trainX = 'CrossTrain/crossreadings.csv'
        testY = 'CrossTest/crossmags.csv'
        testX = 'CrossTest/crossreadings.csv'

        training_data = WindMagDataset(trainY, trainX, transform=None)
        testing_data = WindMagDataset(testY, testX, transform=None)
        epochs = epochs0

    elif dataSetType==5:
        trainY = 'CrossTrain/crossangsrad.csv'
        trainX = 'CrossTrain/crossreadings.csv'
        testY = 'CrossTest/crossangsrad.csv'
        testX = 'CrossTest/crossreadings.csv'

        training_data = WindAngDataset(trainY, trainX, transform=None)
        testing_data = WindAngDataset(testY, testX, transform=None)
        epochs = epochs0

        fullAnglesVal = True

    else:
        raise ValueError('Not a valid dataSetType index (must be in {1, 2, 3, 4, or 5})')

    '''
    # Make training and testing data
    train_dataloader = DataLoader(training_data, batch_size=180, shuffle=True)
    test_dataloader = DataLoader(testing_data, batch_size=72, shuffle=True)
    model = NeuralNetwork(crosswire=crosswireVal, fullAngles=fullAnglesVal, geom=geometryVal)
    opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
    '''
    
    return training_data, testing_data, epochs, fullAnglesVal, crosswireVal, trainPath, trainLabelPath, testPath, testLabelPath, geomPath

In [ ]:
class customLoss():
    def __init__(self):
        self.pi = math.pi
        
    def forward(self, y, yhat):
        self.Errs = torch.cat((torch.remainder(y-yhat, 2.0*self.pi), torch.sub(torch.remainder(y-yhat, 2.0*self.pi), 2.0*self.pi)), 1)
        tmp1 = torch.min(torch.abs(self.Errs), 1).values
        return torch.sum(tmp1)
        # return torch.sum(torch.min(torch.abs(torch.tensor([self.Errs, self.Errs-2.0*math.pi])), 1))



In [ ]:
# Define needed learning quantities

# Learning rate (initial)
# ### Generally ~ 1e-3 for Adam
learning_rate = 1e-3

# Batch size (default 64)
batch_size = 180

# Number of training epochs for 
# ### the simpler regression problems
epochs0 = 20

# Loss Function (MSE/MAE usually because we are 
# running relatively standard regression)

### ### Mean Squared error loss
# loss_fn = nn.MSELoss()

# Mean Absolute Error Loss
# loss_fn = nn.L1Loss()

# Mean Absolute Error Loss **wrapped on angles**
loss_fn = customLoss()

# Verbose flag toggles training
verboseFlag=False

# training_data, testing_data, epochs, fullAnglesVal, crosswireVal, trainPath, testPath = makeDataset(dataSetType=2, geometryVal=3, compFlag=True, N=1)

In [ ]:
NN = np.array([1, 2, 5])
GEOMVAL = np.arange(5, 7)

nFilt = len(NN)
nGeom = len(GEOMVAL)
nSeed = 5
tic()
for jj in range(len(GEOMVAL)):
    for ii in range(len(NN)):
        bestPerformance = 100.0
        N = NN[ii]
        geometryVal = GEOMVAL[jj]
        for kk in range(nSeed):
            np.random.seed(kk*12345 + 31415*N*(geometryVal**3))
            # Make all necessary data
            training_data, testing_data, epochs, fullAnglesVal, crosswireVal, trainPath, trainLabelPath, testPath, testLabelPath, geomPath = makeDataset(dataSetType=2, geometryVal=geometryVal, compFlag=True, compString='compTest', N=N, epochs0=epochs0)
            
            # Make training and testing dataloaders
            # Initialize model and optimizer params
            train_dataloader = DataLoader(training_data, batch_size=180, shuffle=True)
            test_dataloader = DataLoader(testing_data, batch_size=72, shuffle=True)
            test_dataloader2 = DataLoader(testing_data, batch_size=1, shuffle=True)
            model = AngleNet(geom=geometryVal)
            opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
            
            print(f"Epoch {0}\n-------------------------------")
            avg_error = test_loop(test_dataloader, model, 0, kk, loss_fn, lastLoop=False, writePath=(testPath+geomPath))
            
            
            for t in range(1, epochs+1):
                print(f"Epoch {t}\n-------------------------------")
                train_loop(train_dataloader, model, opt, t, kk, loss_fn, verbose=verboseFlag, writePath=(testPath+geomPath))
                print()
                # if (float(t+1)/float(epochs) >= k or (t==(epochs-1))):
                if t < epochs:
                    avg_error = test_loop(test_dataloader, model, t, kk, loss_fn, lastLoop=False, writePath=(testPath+geomPath))
                else: 
                    # avg_error, Z = test_loop(test_dataloader, model, loss_fn, lastLoop=(t==(epochs-1)))
                    avg_error, Z = test_loop(test_dataloader2, model, t, kk, loss_fn, lastLoop=True, writePath=(testPath+geomPath))
            
            if avg_error < bestPerformance:
                bestPerformance = avg_error
                PATH = 'SavedModels/N'+str(N)+'_G'+str(geometryVal)+'_best.tar'
                torch.save({
                        'epoch': epochs,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': opt.state_dict(),
                        'loss': avg_error*180.0/math.pi,
                        }, PATH)
            # Print that we have finished training
            print(f"Finished seed: {kk+1:>2d} of "+str(nSeed)+f", on geometry {jj+1:>2d} of "+str(len(GEOMVAL))+f", on filtering setting {ii+1:>2d} of "+str(len(NN))+" \n")

            
# Output time elapsed in seconds            
dT = toc()
dT = np.round_(dT)
dT2 = dT % 3600
dT3 = dT2 % 60
print(f"Elapsed time is {dT} seconds")
print(f"This is equivalent to {dT // 3600} hours, "+f"{dT2 // 60} minutes, and "+f"{dT3} seconds")


In [ ]:
# Triangle -- Best: ~20.6 degrees mean error; ~30 epochs; network layers 3-30-15-1; 
#          -- tested +10 more epochs and stalled in 20.9-21.5 test error range
#
# Square   -- ~4.5 degrees mean error; ~330 epochs; network layers [4]-50-25-1
#          -- tested +00 more epochs, stalled in 6-6.5 range
#

In [ ]:
# Notes for saving and loading Pytorch Models, from 
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
'''
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()
'''
# Useful reference for saving off weights, etc, that we might need and/or want

In [ ]:
### Data processing section 
# Order in bigData is [tri -- squ -- pent -- hex]
processData = False

# Safety mechanism to not override data
if processData:
    testPathBase='compTest'
    NN = np.array([1, 2, 5])
    lenTraj = 21
    nFilt = 3
    nGeom = 4
    nSeed = 5
    BIGDAT = np.zeros((nFilt*nGeom*nSeed*lenTraj, 5))
    for ii in range(nFilt):
        for jj in np.arange(3, 3+nGeom):
            geometryVal = jj
            for kk in range(nSeed):
                if geometryVal == 3:
                    geomPath='tri/'
                    geoString = 'TRI'
                elif geometryVal == 4:
                    geomPath='squ/'
                    geoString = 'SQU'
                elif geometryVal == 5:
                    geomPath='pent/'
                    geoString = 'PENT'
                elif geometryVal == 6:
                    geomPath='hex/'
                    geoString = 'HEX'
                else:
                    raise ValueError('Geometry must be in {3, 4, 5, 6}')

                testPath=testPathBase+'_N'+str(NN[ii])+'/' + geomPath + 'testCost'+str(kk)+'.csv'
                # testPathInit = 'compVal_N'+str(NN[ii])+'/' + geomPath + 'testCostInit'+str(kk)+'.csv'

                tmp = np.zeros((lenTraj, 5))
                # tmp[0,:2] = pd.read_csv(testPathInit, header=None).to_numpy()
                tmp[0:,:2] = pd.read_csv(testPath, header=None).to_numpy()
                # tmp[1:,0] += 1
                tmp[:,2] = np.ones(lenTraj)*(kk+1)
                tmp[:,3] = np.ones(lenTraj)*geometryVal
                tmp[:,4] = np.ones(lenTraj)*NN[ii]

                BIGDAT[(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + kk*lenTraj):(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + (kk+1)*lenTraj), :] = tmp

    # Save to npy file
    np.save('SummaryStats/BigData.npy', BIGDAT)
else: 
    raise ValueError('Did not process the data due to the processData flag being False. Make sure you want to process the data before proceeding!')

In [ ]:
# Process testErrs into a dataframe and condense to averages
# Order in bigData is [tri -- squ -- pent -- hex]

# Safety mechanism to not override data
if processData:
    
    testPathBase='compTest'
    NN = np.array([1, 2, 5])
    lenDat = 287999
    nFilt = len(NN)
    nGeom = 4
    nSeed = 5
    dfMain = pd.DataFrame(columns=('Angle_deg', 'Err_deg', 'N', 'Geometry'))
    dfMain2 = pd.DataFrame(columns=('Angle_deg', 'Err_deg', 'N', 'Geometry'))
    for ii in range(nFilt):
        for jj in np.arange(3, 3+nGeom):
            geometryVal = jj
            tmpDAT = np.zeros((nSeed*lenDat, 5))
            bestSeed = 900.0*np.ones(nSeed)
            for kk in range(nSeed):
                if geometryVal == 3:
                    geomPath='tri/'
                    geoString = 'TRI'
                elif geometryVal == 4:
                    geomPath='squ/'
                    geoString = 'SQU'
                elif geometryVal == 5:
                    geomPath='pent/'
                    geoString = 'PENT'
                elif geometryVal == 6:
                    geomPath='hex/'
                    geoString = 'HEX'
                else:
                    raise ValueError('Geometry must be in {3, 4, 5, 6}')

                trainPath = testPathBase+'_N'+str(NN[ii])+'/' + geomPath + 'trainCost'+str(kk)+'.csv' 
                testPath = testPathBase+'_N'+str(NN[ii])+'/' + geomPath + 'testErrs'+str(kk)+'.csv'
                # testPathInit = 'compVal_N'+str(NN[ii])+'/' + geomPath + 'testCostInit'+str(kk)+'.csv'

                tmp0 = pd.read_csv(trainPath, header=None).to_numpy()
                bestSeed[kk] = tmp0[-1, 1]
                # print(tmp0[-1, 1])

                tmp = np.zeros((lenDat, 5))
                # tmp[0,:2] = pd.read_csv(testPathInit, header=None).to_numpy()
                tmp[:,:2] = pd.read_csv(testPath, header=None).to_numpy()
                tmp[:,:2] *= 180.0/math.pi
                tmp[:,0] = np.round_(tmp[:,0])
                # bestSeed[kk] = np.mean(tmp[:,1])
                tmp[:,2] = np.ones(lenDat)*NN[ii]
                tmp[:,3] = np.ones(lenDat)*geometryVal
                tmp[:,4] = np.ones(lenDat)*(kk+1)
                tmpDAT[kk*lenDat:(kk+1)*lenDat,:] = tmp


            # Choose the best seed for each geometry-filter configuration
            kkStar = np.argmin(bestSeed)
            # print(bestSeed)
            # print(f"The best seed is Seed {kkStar+1}, representing element"+ f" [{kkStar}] of bestSeed array")
            # breakpoint()

            dfTmp = pd.DataFrame(data=tmpDAT[kkStar*lenDat:(kkStar+1)*lenDat,:4],
                                 columns=('Angle_deg', 'Err_deg', 'N', 'Geometry')
                                )

            dfMain = pd.concat([dfMain, dfTmp])

            dfTmp2 = dfTmp.groupby(['Angle_deg'], as_index=False).mean()
            # print(dfTmp2.shape)
            # Append to main dataframe
            dfMain2 = pd.concat([dfMain2, dfTmp2])
            # print(dfMain.shape)


            # BIGDAT[(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + kk*lenTraj):(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + (kk+1)*lenTraj), :] = tmp

    # Save to csv file
    dfMain.to_csv('SummaryStats/ErrorVSAngleLarge.csv')
    dfMain2.to_csv('SummaryStats/ErrorVSAngle.csv')

else: 
    raise ValueError('Did not process the data due to the processData flag being False. Make sure you want to process the data before proceeding!')

In [ ]:
Zs = np.sort(Z)
np.savetxt(testPath+geomPath+'ValErrorSort.csv', Zs, delimiter=',')

print('Mean Absolute Error (deg): ', str(np.mean(Zs)*180.0/math.pi))

# Choose what fraction of Zs to study
fracPred = 0.98


# Take the >fracPred set of best predictions
nKeep = int(np.ceil(fracPred*len(Zs)))
Zsmall = Zs[:nKeep]

# Print the fracPred quantile worst prediction
print(np.max(Zsmall))
print('This is equivalent to '+ str(np.max(Zsmall)*180.0/math.pi) + ' degrees')
print('Mean Absolute Error, best 98% (deg): ', str(np.mean(Zsmall)*180.0/math.pi))

In [ ]:
plt.figure(2)
plt.hist(Zsmall, cumulative=False, density=True, bins=20)
plt.show()

In [ ]:
zs = int(len(Zsmall)*0.84)
print(zs)

In [ ]:
print(Zsmall.shape)
print(Zsmall[zs]*180/math.pi)

In [ ]:
NN = np.array([1, 2, 5])
print(NN[0])
print(NN.shape)
GEOMVAL = np.arange(3, 7)
print(GEOMVAL)
print(GEOMVAL.shape)
# training_data, testing_data, epochs, fullAnglesVal, crosswireVal, trainPath, testPath = makeDataset(dataSetType=2, geometryVal=3, compFlag=True, N=1)

In [ ]:


'''
# Make dataset
# Change this as desired in {1, 2, 3, 4, 5}
#
# INDEX: 
#   --- (1) Sparse wind magnitudes                         [OLD]
#   --- (2) Sparse wind angles (10-degree increments)      [OLD]
#   --- (3) Dense Crosswire Model 
#   --- (4) Dense Wind Magnitudes 
#   --- (5) Dense Incidence Angles (2-degree increments)
dataSetType = 2

# 3=Triangle, 4=Square, 5=Pentagon, 6=Hexagon
geometryVal = 3
compFlag=True
N = 1                 # Number of sequentially averaged data points

if geometryVal == 3:
    geomPath='tri/'
elif geometryVal == 4:
    geomPath='squ/'
elif geometryVal == 5:
    geomPath='pent/'
elif geometryVal == 6:
    geomPath='hex/'
else:
    raise ValueError('Geometry must be in {3, 4, 5, 6}')

trainPath='compTrain_N'+str(N)+'/'
testPath='compVal_N'+str(N)+'/'       # Set to validation data for network/hyperparameter optimization, else test data

# Don't change these; the 'if' statements take care of them
# Set network parameters in NeuralNetwork class
fullAnglesVal = False
crosswireVal = False

if dataSetType==1:
    if compFlag:
        trainY = trainPath+geomPath+'mags.csv'
        trainX = trainPath+geomPath+'readings.csv'
        testY = testPath+geomPath+'mags.csv'
        testX = testPath+geomPath+'readings.csv'
    else:
        trainY = 'MagTrain/mags.csv'
        trainX = 'MagTrain/readings.csv'
        testY = 'MagTest/mags.csv'
        testX = 'MagTest/readings.csv'
    
    training_data = WindMagDataset(trainY, trainX, transform=None)
    testing_data = WindMagDataset(testY, testX, transform=None)
    epochs = epochs0
    
elif dataSetType==2:
    if compFlag:
        trainY = trainPath+geomPath+'angsrad.csv'
        trainX = trainPath+geomPath+'readings.csv'
        testY = testPath+geomPath+'angsrad.csv'
        testX = testPath+geomPath+'readings.csv'
        fullAnglesVal = True
        
    else:
        trainY = 'MagTrain/angsrad.csv'
        trainX = 'MagTrain/readings.csv'
        testY = 'MagTest/angsrad.csv'
        testX = 'MagTest/readings.csv'
    
    training_data = WindAngDataset(trainY, trainX, transform=None)
    testing_data = WindAngDataset(testY, testX, transform=None)
    epochs = epochs0
    
elif dataSetType==3:
    trainY1 = 'CrossTrain/crossmags.csv'
    trainY2 = 'CrossTrain/crossangsrad.csv'
    trainX = 'CrossTrain/crossreadings.csv'
    testY1 = 'CrossTest/crossmags.csv'
    testY2 = 'CrossTest/crossangsrad.csv'
    testX = 'CrossTest/crossreadings.csv'
    
    training_data = CrossWireDataset(trainY1, trainY2, trainX, transform=None)
    testing_data = CrossWireDataset(testY1, testY2, testX, transform=None)
    epochs = 2*epochs0
    
    crosswireVal = True
    
elif dataSetType==4:
    
    trainY = 'CrossTrain/crossmags.csv'
    trainX = 'CrossTrain/crossreadings.csv'
    testY = 'CrossTest/crossmags.csv'
    testX = 'CrossTest/crossreadings.csv'
    
    training_data = WindMagDataset(trainY, trainX, transform=None)
    testing_data = WindMagDataset(testY, testX, transform=None)
    epochs = epochs0
    
elif dataSetType==5:
    trainY = 'CrossTrain/crossangsrad.csv'
    trainX = 'CrossTrain/crossreadings.csv'
    testY = 'CrossTest/crossangsrad.csv'
    testX = 'CrossTest/crossreadings.csv'
    
    training_data = WindAngDataset(trainY, trainX, transform=None)
    testing_data = WindAngDataset(testY, testX, transform=None)
    epochs = epochs0
    
    fullAnglesVal = True
    
else:
    raise ValueError('Not a valid dataSetType index (must be in {1, 2, 3, 4, or 5})')
   
'''
'''
# Make training and testing data
train_dataloader = DataLoader(training_data, batch_size=180, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=72, shuffle=True)
model = NeuralNetwork(crosswire=crosswireVal, fullAngles=fullAnglesVal, geom=geometryVal)
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
'''